In [5]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

service = Service(executable_path=r'C:\Users\DELL\Documents\web_scraping\chromedriver-win64\chromedriver.exe')
driver = webdriver.Chrome(service=service)
driver.get("https://www.airbnb.com/")
driver.maximize_window()

element_to_click = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//div[text()="Where"]'))
)
element_to_click.click()

# Locate search input, click, write, and press Enter
search_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "bigsearch-query-location-input"))
)
search_input.click()
search_input.send_keys("Germany")
search_input.send_keys(Keys.ENTER)

# Locate and click search button
search_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'button[data-testid="structured-search-input-search-button"]'))
)
search_button.click()
time.sleep(3)

show_filters_button = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'button[data-testid="category-bar-filter-button"]'))
)
show_filters_button.click()

popup = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.XPATH, '//div[@aria-label="Filters"]'))
)
driver.execute_script("arguments[0].scrollIntoView();", popup)

# Locate and click "Room"
room_span = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.XPATH, '//span[text()="Room"]'))
)
room_span.click()

# Locate and adjust slider value
slider = driver.find_element(By.CSS_SELECTOR, 'input[role="slider"]')
ActionChains(driver).click_and_hold(slider).move_by_offset(50, 0).release().perform()

# Locate and click "show" in footer
try:
    show_link = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//a[contains(text(),"Show")]'))
    )
    show_link.click()
except TimeoutException:
    print("Error: 'Show' link not found or not clickable within timeout.")

address = []
room_descriptions = []
price = []
host_rating = []

def extract_data():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    card_containers = driver.find_elements(By.XPATH, "//div[@data-testid='card-container']")
    print(f"Found {len(card_containers)} cards")

    for card in card_containers:
        # card Title
        card_title_elements = card.find_elements(By.XPATH, './/div[@data-testid="listing-card-title"]')
        if card_title_elements:
            address.append(card_title_elements[0].text.strip())
            print(f"Address: {card_title_elements[0].text.strip()}")
        else:
            address.append('None')
            print("Address: None")

        # Description
        desc_elements = card.find_elements(By.XPATH, './/span[@data-testid="listing-card-name"]')
        if desc_elements:
            room_descriptions.append(desc_elements[0].text.strip())
            print(f"Description: {desc_elements[0].text.strip()}")
        else:
            room_descriptions.append('None')
            print("Description: None")

        # Stars
        price_elements = card.find_elements(By.XPATH, './/div//span[contains(text(),"per")]')
        if price_elements:
            price.append(price_elements[0].text.strip())
            print(f"Price: {price_elements[0].text.strip()}")
        else:
            price.append('None')
            print("Price: None")

        # Forks
        host_elements = card.find_elements(By.XPATH, './/div//span[contains(text(),"reviews")]')
        if host_elements:
            host_rating.append(host_elements[0].text.strip())
            print(f"Host Rating: {host_elements[0].text.strip()}")
        else:
            host_rating.append('None')
            print("Host Rating: None")

while True:
    extract_data()
    try:
        
        next_button = driver.find_element(By.XPATH, '//a[@aria-label="Next"]')
        if next_button.is_enabled():
            driver.execute_script("arguments[0].scrollIntoView();", next_button)
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(3)  
        else:
            break
    except NoSuchElementException:
        break
    except ElementClickInterceptedException:
        driver.execute_script("arguments[0].scrollIntoView();", next_button)
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(3)  # Wait for the next page to load


df = pd.DataFrame({
    'Address': address,
    'Description': room_descriptions,
    'Price': price,
    'Host Rating': host_rating
})

driver.quit()
df.to_csv('airbnb.csv', index=False)
print(df)

Found 18 cards
Address: Room in Behren-Lübchin
Description: Shepherd room manor house Viecheln Anno 1869
Price: $62 per night
Host Rating: 4.92 out of 5 average rating, 139 reviews
Address: Room in Schleusingen
Description: Casa Cabrera
Price: $42 per night
Host Rating: 4.98 out of 5 average rating, 189 reviews
Address: Room in Bad Herrenalb
Description: Room 6 guest rooms in the North Black Forest
Price: $24 per night
Host Rating: 4.87 out of 5 average rating, 192 reviews
Address: Room in Tecklenburg
Description: Völlinck am Habichtswald
Price: $38 per night
Host Rating: 4.99 out of 5 average rating, 240 reviews
Address: Room in Langenhorn
Description: Idyll + Breakfast anywhere in NFlands
Price: $52 per night
Host Rating: 4.98 out of 5 average rating, 261 reviews
Address: Room in Oberkirch
Description: HappyFarm - sleeping in nature
Price: $53 per night
Host Rating: 4.93 out of 5 average rating, 172 reviews
Address: Room in Berlin
Description: Berlin-Grunewald feel-good room
Price: $